In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170916.txt'
df = pd.read_csv(url)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198419 entries, 0 to 198418
Data columns (total 11 columns):
C/A                                                                     198419 non-null object
UNIT                                                                    198419 non-null object
SCP                                                                     198419 non-null object
STATION                                                                 198419 non-null object
LINENAME                                                                198419 non-null object
DIVISION                                                                198419 non-null object
DATE                                                                    198419 non-null object
TIME                                                                    198419 non-null object
DESC                                                                    198419 non-null object
ENTRIES                           

In [3]:
# REMOVE NON-REGULAR DESC
# REMOVE PATH STATIONS in New Jersey

#Jan?
# Build scraper for csv's

In [4]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/09/2017,00:00:00,REGULAR,6318862,2138544
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/09/2017,04:00:00,REGULAR,6318888,2138549
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/09/2017,08:00:00,REGULAR,6318905,2138584
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/09/2017,12:00:00,REGULAR,6318985,2138669
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/09/2017,16:00:00,REGULAR,6319212,2138731


In [5]:
## Create DATETIME column from 'DATE' and 'TIME' cols
df['DATETIME'] = df[['DATE','TIME']].apply(
    lambda x: datetime.datetime.strptime(
        '{0} {1}'.format(x[0],x[1]), 
        '%m/%d/%Y %H:%M:%S'), 
    axis=1)

In [6]:
## STANDARDIZE COL NAMES
df.columns = df.columns.str.strip()

## DROP UNNECESSARY COLUMNS
df_2 = df.drop(['LINENAME', 'DIVISION', 'DATE', 'TIME', 'DESC'], axis=1)

In [7]:
## create diff columns to take entries/exits for given time interval
df_2['entry_diff'] = df_2.sort_values(['STATION', 'C/A', 'UNIT', 'SCP','DATETIME']).groupby(['STATION', 'C/A', 'UNIT', 'SCP'])['ENTRIES'].diff()
df_2['exit_diff'] = df_2.sort_values(['STATION', 'C/A', 'UNIT', 'SCP','DATETIME']).groupby(['STATION', 'C/A', 'UNIT', 'SCP'])['EXITS'].diff()
df_2.head()

,C/A,UNIT,SCP,STATION,ENTRIES,EXITS,DATETIME,entry_diff,exit_diff
0,A002,R051,02-00-00,59 ST,6318862,2138544,2017-09-09 00:00:00,NaN,NaN
1,A002,R051,02-00-00,59 ST,6318888,2138549,2017-09-09 04:00:00,26.0,5.0
2,A002,R051,02-00-00,59 ST,6318905,2138584,2017-09-09 08:00:00,17.0,35.0
3,A002,R051,02-00-00,59 ST,6318985,2138669,2017-09-09 12:00:00,80.0,85.0
4,A002,R051,02-00-00,59 ST,6319212,2138731,2017-09-09 16:00:00,227.0,62.0


In [8]:
#time deltas

In [9]:
df_2 = df_2.groupby(['STATION', 'C/A', 'UNIT', 'SCP',pd.Grouper(key='DATETIME', freq='4H')]).sum()
df_2

ENTRIES     EXITS  \
STATION   C/A  UNIT SCP      DATETIME                                  
1 AV      H007 R248 00-00-00 2017-09-09 00:00:00  12695356  14074548   
                             2017-09-09 04:00:00  12695459  14074898   
                             2017-09-09 08:00:00  12695565  14075178   
                             2017-09-09 12:00:00  12696039  14076012   
                             2017-09-09 16:00:00  12696759  14077114   
                             2017-09-09 20:00:00  12697521  14078327   
                             2017-09-10 00:00:00  12697983  14079224   
                             2017-09-10 04:00:00  12698110  14079600   
                             2017-09-10 08:00:00  12698162  14079801   
                             2017-09-10 12:00:00  12698571  14080388   
                             2017-09-10 16:00:00  12699147  14081368   
                             2017-09-10 20:00:00  12699623  14082328   
                             2017-09-11 00:00:00  12699955  14082912   
                             2017-09-11 04:00:00  12700006  14083016   
                             2017-09-11 08:00:00  12700376  14083774   
                             2017-09-11 12:00:00  12701369  14085440   
                             2017-09-11 16:00:00  12702189  14086331   
                             2017-09-11 20:00:00  12703223  14087300   
                             2017-09-12 00:00:00  12703574  14087860   
                             2017-09-12 04:00:00  12703592  14087952   
                             2017-09-12 08:00:00  12703975  14088913   
                             2017-09-12 12:00:00  12705060  14090399   
                             2017-09-12 16:00:00  12706046  14091221   
                             2017-09-12 20:00:00  12707085  14092180   
                             2017-09-13 00:00:00  12707448  14092777   
                             2017-09-13 04:00:00  12707491  14092883   
                             2017-09-13 08:00:00  12707834  14093874   
                             2017-09-13 12:00:00  12708977  14095350   
                             2017-09-13 16:00:00  12709900  14096192   
                             2017-09-13 20:00:00  12710966  14097078   
...                                                    ...       ...   
ZEREGA AV R419 R326 00-05-01 2017-09-11 00:00:00        39        78   
                             2017-09-11 04:00:00        39        78   
                             2017-09-11 08:00:00        39        78   
                             2017-09-11 12:00:00        39        78   
                             2017-09-11 16:00:00        39        78   
                             2017-09-11 20:00:00        39        78   
                             2017-09-12 00:00:00        39        78   
                             2017-09-12 04:00:00        39        78   
                             2017-09-12 08:00:00        39        78   
                             2017-09-12 12:00:00        39        78   
                             2017-09-12 16:00:00        39        78   
                             2017-09-12 20:00:00        39        78   
                             2017-09-13 00:00:00        39        78   
                             2017-09-13 04:00:00        39        78   
                             2017-09-13 08:00:00        39        78   
                             2017-09-13 12:00:00        39        78   
                             2017-09-13 16:00:00        39        78   
                             2017-09-13 20:00:00        39        78   
                             2017-09-14 00:00:00        39        78   
                             2017-09-14 04:00:00        39        78   
                             2017-09-14 08:00:00        39        78   
                             2017-09-14 12:00:00        39        78   
                             2017-09-14 16:00:00        39        78   
                   

In [10]:
## ASSUMPTIONS:
## 1. All negative actuals for entries and exits will be dropped from dataset
## 2. All outliers (+/- 1.5*IQR) for entries and exits will be dropped from dataset

len_entry_diffs = df_2['entry_diff'].shape[0]
len_exit_diffs = df_2['exit_diff'].shape[0]

negative_entries = df_2[df_2['entry_diff'] < 0].shape[0] / len_entry_diffs
negative_exits = df_2[df_2['exit_diff'] < 0].shape[0] / len_exit_diffs

print('negative_entries: ', negative_entries)
print('negative_exits: ', negative_exits)

## DROP NEGATIVES
df_2 = df_2[(df_2['entry_diff'] > 0) | (df_2['exit_diff'] > 0)]

negative_entries:  0.00803255918459056
negative_exits:  0.006832526502849439


In [11]:
len_entry_diffs = df_2['entry_diff'].shape[0]
len_exit_diffs = df_2['exit_diff'].shape[0]

entry_IQR = df_2['entry_diff'].quantile(0.75) - df_2['entry_diff'].quantile(0.25)
exit_IQR = df_2['exit_diff'].quantile(0.75) - df_2['exit_diff'].quantile(0.25)
cutoff_entry = df_2['entry_diff'].quantile(0.75) + entry_IQR * 3
cutoff_exit = df_2['exit_diff'].quantile(0.75) + exit_IQR * 3
print(cutoff_entry)
print(cutoff_exit)

outliers_entry = df_2[df_2['entry_diff'] > cutoff_entry].shape[0] / len_entry_diffs
outliers_exit = df_2[df_2['exit_diff'] > cutoff_exit].shape[0] / len_exit_diffs

print('outliers_entry: ', outliers_entry)
print('outliers_exit: ', outliers_exit)

## DROP OUTLIERS
df_2 = df_2[(df_2['entry_diff'] < outliers_entry) | (df_2['exit_diff'] < outliers_exit)]

1123.0
757.0
outliers_entry:  0.017117396913373786
outliers_exit:  0.03284915667255961


In [12]:
df_2.head()

ENTRIES    EXITS  entry_diff  \
STATION C/A  UNIT SCP      DATETIME                                            
1 AV    H007 R248 00-03-02 2017-09-12 04:00:00  5692969   476676        16.0   
                           2017-09-15 04:00:00  5696903   477005        19.0   
103 ST  R252 R180 00-03-01 2017-09-11 00:00:00  1039334  1852394         0.0   
                           2017-09-11 04:00:00  1039334  1852443         0.0   
                           2017-09-11 08:00:00  1039334  1853029         0.0   

                                                exit_diff  
STATION C/A  UNIT SCP      DATETIME                        
1 AV    H007 R248 00-03-02 2017-09-12 04:00:00        0.0  
                           2017-09-15 04:00:00        0.0  
103 ST  R252 R180 00-03-01 2017-09-11 00:00:00      253.0  
                           2017-09-11 04:00:00       49.0  
                           2017-09-11 08:00:00      586.0

In [13]:
## NEXT STEPS

# 1. RENKOH: make mockup weather model based on random weather data
# 2. Create Day of the Week column and week/weekend column - used to segregate shortlist
# 3. Request weather data from API